In [1]:
import os
import sys
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
import tqdm
import utils as utils
import scienceplots
plt.style.use('science')

base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

import fit_FaIRGP as fairgp
import fit_GP as plaingp
import evaluate_FaIRGP as evalFaIRGP
import evaluate_GP as evalGP

<IPython.core.display.Javascript object>

In [3]:
train_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['historical']}}
train_data = plaingp.make_data(train_cfg)

test_cfg = {'dataset' : {'dirpath': '../data/', 'keys': ['ssp126', 'ssp245', 'ssp370', 'ssp585']}}
test_data = plaingp.make_data(test_cfg)

gpmodel = plaingp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/PlainGP/state_dict.pt')
gpmodel.load_state_dict(state_dict)

fairgpmodel = fairgp.make_model(train_cfg, train_data)
state_dict = torch.load('../data/models/historical-only/FaIRGP/state_dict.pt')
fairgpmodel.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
test_scenarios = test_data.scenarios
test_times = test_scenarios.timesteps
test_tas = test_scenarios.tas
test_tas_fair = fairgpmodel._compute_mean(test_scenarios)
X_test = torch.cat([test_scenarios.cum_emissions[:, 0, None],
                    test_scenarios.emissions[:, 1:]], dim=-1)
X_test = (X_test - gpmodel.mu) / gpmodel.sigma

gpmodel = gpmodel.eval()
fairgpmodel = fairgpmodel.eval()

with torch.no_grad():
    noisy_gp_test_posterior = gpmodel.likelihood(gpmodel(X_test))
    mean = gpmodel.sigma_targets * noisy_gp_test_posterior.mean + test_tas_fair
    covar = gpmodel.sigma_targets.pow(2) * noisy_gp_test_posterior.covariance_matrix
    noisy_gp_test_posterior = noisy_gp_test_posterior.__class__(mean, covar)
    
with torch.no_grad():
    fairgp_test_posterior = fairgpmodel(test_scenarios)
    noisy_fairgp_test_posterior = fairgpmodel.likelihood(fairgp_test_posterior)
    mean = noisy_fairgp_test_posterior.mean + test_tas_fair
    covar = noisy_fairgp_test_posterior.lazy_covariance_matrix
    noisy_fairgp_test_posterior = noisy_fairgp_test_posterior.__class__(mean, covar)

/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/lazy_tensor.py:25: UserWarning: gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense
  warnings.warn("gpytorch.lazy.delazify is deprecated in favor of linear_operator.to_dense")
/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/gpytorch/lazy/non_lazy_tensor.py:13: UserWarning: gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator
  warnings.warn("gpytorch.lazy.lazify is deprecated in favor of linear_operator.to_linear_operator")


In [14]:
gp_scores = evalGP.compute_scores(noisy_gp_test_posterior, test_scenarios)
fairgp_scores = evalFaIRGP.compute_scores(noisy_fairgp_test_posterior, test_scenarios)

In [15]:
gp_scores

{'mb': 0.11039731651544571,
 'rmse': 0.2654469609260559,
 'mae': 0.20985470712184906,
 'corr': 0.9666864480692011,
 'll': -0.24941770732402802,
 'calib95': 0.9156976938247681,
 'ICI': 0.0352203135819812}

In [16]:
fairgp_scores

{'mb': -0.07356604933738708,
 'rmse': 0.19910657405853271,
 'mae': 0.14806316792964935,
 'corr': 0.9760644133624948,
 'll': 0.14821457862854004,
 'calib95': 0.8895348906517029,
 'ICI': 0.02585679449533165}